In [7]:
import numpy as np
import scipy.io

def getRelatedDiseases(G_ID, phenotype_network):
    m = phenotype_network.shape[0]
    line_num = 0
    for i in range(m):
        if phenotype_network[i, 0] == int(G_ID):
            line_num = i
            break

    similarity_vec = phenotype_network[line_num, 1:]

    pos = np.argsort(similarity_vec)
    relationDiseasePos = pos[-5:]

    relationDiseaseID = phenotype_network[relationDiseasePos, 0]

    return relationDiseaseID, relationDiseasePos

# 加载ppi_network
ppi_network = scipy.io.loadmat("ppi_network.mat")["ppi_network"]
n = ppi_network.shape[0]  # n是基因的数量 (8919)
outDegree = np.sum(ppi_network, axis=1)
ppi_network = ppi_network / outDegree.reshape(-1, 1)

# 加载phenotype_network
phenotype_network = scipy.io.loadmat("phenotype_network.mat")["phenotype_network"]

# 加载g_p_network
g_p_network = scipy.io.loadmat("g_p_network.mat")["g_p_network"]

# 设置基因ID
G_ID = "1"

relatedDiseaseID, relatedDiseasePos = getRelatedDiseases(G_ID, phenotype_network)

# 初始化优先级向量R
R = np.zeros(n)

for i in range(5):
    pheno = relatedDiseasePos[i]
    for j in range(n):
        if g_p_network[j, pheno] != 0:
            R[j] = 1

if np.sum(R) > 0:
    R = R / np.sum(R)

for i in range(n):
    if R[i] != 0:
        print("PR {} is {}".format(i, R[i]))

# 迭代
e = R
iterator = 50
d = 0.1

for _ in range(iterator):
    R = (1 - d) * np.dot(ppi_network, R) + d * e

# 输出相关基因
t = np.argsort(R)[::-1]
print("Related genes (top 10):")
for i in range(min(10, len(t))):
    print(f"Gene {t[i]}: {R[t[i]]:.4f}")


PR 455 is 1.0
Related genes (top 10):
Gene 455: 0.1192
Gene 456: 0.0501
Gene 458: 0.0451
Gene 770: 0.0283
Gene 196: 0.0053
Gene 3834: 0.0048
Gene 760: 0.0039
Gene 457: 0.0027
Gene 5090: 0.0027
Gene 3978: 0.0021
